### 1 

Импортируем файлы workouts, payments и users. Преобразуем типы данных "дата+время" 

In [13]:
import pandas as pd
workouts = pd.read_csv('workouts.csv',sep=',', parse_dates = ['start_at'])    # тип столбцов с датами можно сразу переводить правильный тип дата+время. 
payments = pd.read_excel('payments.xlsx')    
users = pd.read_excel('users.xlsx')    
workouts['start_at'] = pd.to_datetime(workouts['start_at']) ## тип данных столбца'start_at' переводим из непонятного "Obgect" в тип дата+время- "datetime64[ns]"
users['first_contact_datetime'] = pd.to_datetime(users['first_contact_datetime']) 
workouts.head(10)  


,workout_id,cost,start_at,status,workout_schedule_type,client_id,client_status,workout_type,trainer_department,trainer_id
0,30793909,NaN,2020-11-13 12:45:00,success,trial,22034,NaN,general,Sales,10722051
1,31123309,750.0,2020-11-21 13:00:00,success,regular,22034,new,general,Spartacus,940642
2,31412167,750.0,2020-11-28 13:00:00,success,regular,22034,new,general,Spartacus,940642
3,31703605,750.0,2020-12-05 13:00:00,success,regular,22034,active,general,Spartacus,940642
4,26904500,NaN,2020-08-06 18:20:00,success,trial,88101,NaN,general,Sales,3826530
5,27205345,750.0,2020-08-15 10:00:00,success,single,88101,new,general,Consultant,10231664
6,27439942,750.0,2020-08-20 19:00:00,success,regular,88101,new,general,Consultant,10231664
7,27698497,750.0,2020-08-27 19:00:00,success,regular,88101,new,general,Consultant,10231664
8,27973407,750.0,2020-09-03 19:00:00,success,regular,88101,active,general,Consultant,10231664
9,28246406,750.0,2020-09-10 19:00:00,success,regular,88101,active,general,Spartacus,10231664


In [12]:
users.dtypes

first_contact_datetime    datetime64[ns]
age                              float64
workouts_successful                int64
workouts_total                     int64
months_active                      int64
user_id                            int64
region                            object
geo_group                         object
dtype: object

### 2

~~~ sql
select    workout_type
        , count(workout_id) as cnt
from workouts
group by workout_type
order by cnt desc
~~~

In [14]:
####
workouts.groupby('workout_type') .agg(cnt = ('workout_id','count')) .reset_index()  .sort_values('cnt', ascending = False) 


,workout_type,cnt
2,general,103492
0,cycling,1175
1,functional test,26


### 3

~~~ sql
select    date_part('month', start_at) as dm
        , trainer_department
        , count(workout_id) as cnt
from workouts
group by   dm
         , trainer_department
order by dm
~~~

In [15]:
####
workouts['start_at'] = pd.to_datetime(workouts['start_at'])                 # к типу дата тут приводить не обязательно, ниже приводит 
workouts['dm'] = workouts['start_at'].dt.to_period('M').dt.to_timestamp()   # DATE TRUNC в питоне (M- обрезать до месяца; Y,M,D, H,T,S...)
   
## Group by, где столбцов-измерений :
workouts.groupby(['dm','trainer_department']) .agg(cnt =('workout_id','count'))  .reset_index()  .sort_values('dm') 


,dm,trainer_department,cnt
0,2020-01-01,Athletic,11
1,2020-01-01,Consultant,164
2,2020-01-01,Cycling,1
3,2020-01-01,Dinamo,36
4,2020-01-01,Sales,457
...,...,...,...
78,2020-12-01,Consultant,4
79,2020-12-01,Cycling,77
80,2020-12-01,Dinamo,254
81,2020-12-01,Sales,22


### 4

~~~ sql
select    date_trunc('month', payment_date) as p_tm
        , sum(amount)/1000 as sum_amt
from payments
where workout_type = 'general'
group by p_tm
order by p_tm
~~~

In [22]:
####
payments['p_tm'] = payments['payment_date'].dt.to_period('M').dt.to_timestamp()   ## DATE TRUNC (M- обрезать до месяца; Y,M,D, H,T,S...)
month_payments = payments [payments['workout_type'] == 'general']  .groupby(['p_tm']) [['amount']].sum()/1000 

month_payments  .reset_index() .sort_values('p_tm') .head(12) 
    # новый датафрейм month_payments создал специально для resed и sort отдельно.


,p_tm,amount
0,2020-01-01,4370.772775
1,2020-02-01,6114.601296
2,2020-03-01,7949.117944
3,2020-04-01,11908.988560
4,2020-05-01,11125.842432
5,2020-06-01,9682.458175
6,2020-07-01,9553.436788
7,2020-08-01,8345.359401
8,2020-09-01,9876.673936
9,2020-10-01,11044.861465


### 5

~~~ sql
select    user_id
        , sum(workouts_successful) as cnt_workouts
from payments p
    join users u
        on p.user_id = u.user_id
where region is not null
group by user_id
order by cnt_workouts desc
limit 20
~~~

In [18]:
####
p_u = payments.merge(users, on = 'user_id', how = 'inner')      
p_u.loc[~p_u['region'].isnull()] \
        .groupby(['user_id']) .agg(cnt_workouts = ('workouts_successful', 'sum')) \
        .reset_index() \
        .sort_values('cnt_workouts', ascending = False) \
        .head(20)
## Очередность такая:  # Джойним отдельно.Джойнутую таблицу называем p_u.
# Фильтруем не нули в поле регион. 
# Где группировка задаём Алиас cnt_workouts для агрегации суммы.
# .reset_index() Это пишем всегда, дабы не ушли индексы.

,user_id,cnt_workouts
3186,3376018,3945
526,329226,2646
881,566918,2156
2278,2316330,2016
322,200400,1890
1444,1380089,1872
458,286678,1752
3292,3524600,1644
1492,1443818,1600
170,116239,1599


### 6

~~~ sql
select    avg(age) as avg_age
from users p
    left join payments u
        on p.user_id = u.user_id
where payment_date is null
~~~

In [19]:
####
# создаём джойнутую таблицу:
new_6 = users.merge(payments, on = 'user_id', how = 'left')

# фильтруем оставляю тех, у кого нуллы в дате платежа; ['age'].mean() - выводим срзнач по столбцу age:
new_6.loc[new_6['payment_date'].isnull()]['age'].mean()      


29.5

### 7

~~~ sql
select   p_tm
       , sum(case when workout_type<>'general' then 1.0 else 0.0 end) / count(payment_id) as share_not_general
from payments
group by p_tm
~~~

In [21]:
####   
payments['p_tm'] = payments['payment_date'].dt.to_period('M').dt.to_timestamp()   ## DATE TRUNC (M- обрезать до месяца; Y,M,D, H,T,S...)
     # Используем библиотеку NUMPY
import numpy as np                                       
payments['bin'] = np.where(payments['workout_type']!='general',1,0)  # бины: 1 - типы тренировки все, кроме general.
payments.head()
    # Создаем 2 колонки через агрегации: сумма бинов и кол-во платежей (сохраняем в новой таблице):
payments_othertypes = payments.groupby(['p_tm']) .agg(cnt_others = ('bin' , 'sum') , cnt_pay = ('payment_id','count')).reset_index()  
    # Создаём еще одну колонку: 
payments_othertypes['share_others'] = payments_othertypes['cnt_others'] / payments_othertypes['cnt_pay']
    # Выбираем что нужно по условию (месяц и доля всех, кроме general):
payments_othertypes[['p_tm','share_others']]


,p_tm,share_others
0,2020-01-01,0.004255
1,2020-02-01,0.012085
2,2020-03-01,0.006969
3,2020-04-01,0.007009
4,2020-05-01,0.003317
5,2020-06-01,0.010436
6,2020-07-01,0.007656
7,2020-08-01,0.007642
8,2020-09-01,0.017479
9,2020-10-01,0.017843


### 8

~~~ sql
select    w.clientId
        , cnt_workout
        , amt
from (select    client_id
              , count(workout_id) as cnt_workout
     from workouts
     group by client_id
     ) w
    join (select      user_id
                    , sum(amount) as amt
          from payments
          group by user_id
          ) p
        on p.user_id = w.client_id
limit 10
~~~

In [24]:
####
#Подзапросы оформляем по шагам - отдельные таблички кажного селекта, а потом их джойним -merge:
w = workouts.groupby('client_id').agg(cnt_workout = ('workout_id','count')).reset_index() 
p = payments.groupby('user_id').agg(amt = ('amount','sum')).reset_index()  ##опять косячный формат суммы, к-й я не умею переводить в int.

#JOIN когда столбцы-ключи названы по разному:
basic_client_info = w.merge(p, how = 'inner', left_on=['client_id'], right_on=['user_id']) 

basic_client_info[['client_id','cnt_workout','amt']] .sort_values('cnt_workout', ascending = False) .head(10)

,client_id,cnt_workout,amt
3712,3376018,264,149430.813232
4165,4487733,179,42666.687058
351,198891,152,60750.829718
3422,3011832,146,50546.338203
3997,3917813,139,51656.419723
545,297451,137,41721.508862
442,241469,131,57281.343942
4170,4511817,125,88714.942116
2606,2255913,124,22032.755015
312,181047,116,84751.604905


In [ ]:
################################################################################################################################
################################################################################################################################
################################################################################################################################